In [32]:
using HTTP
using JSON3
using JSON
using Dates
using NCDatasets
using OrderedCollections
using Colors
using Makie
using CairoMakie
using GeoMakie
using DIVAnd
using Random
include("./DIVAndFairEase.jl")
using JupyterFormatter
enable_autoformat()

2-element Vector{Function}:
 format_current_cell (generic function with 1 method)
 format_current_cell (generic function with 1 method)

In [2]:
token = ENV["beaconAPItoken"];
regionname = "North_Adriatic"
varname = "sea_water_temperature"
doplot = false

datasourcelist = [
    "Euro-Argo",
    "CORA Profile",
    "CORA Timeseries",
    "World Ocean Database",
    "SeaDataNet CDI TS",
]

5-element Vector{String}:
 "Euro-Argo"
 "CORA Profile"
 "CORA Timeseries"
 "World Ocean Database"
 "SeaDataNet CDI TS"

In [3]:
datadir = joinpath("../data/", regionname)
outputdir = joinpath("../output/", regionname)
figdir = joinpath("../figures/", regionname)
mkpath.([datadir, outputdir, figdir]);

In [4]:
domaininfo = OrderedDict(
    "North_Adriatic" => [12.0, 18.0, 43.0, 46.0],
    "Arctic_region" => [-44.25, 70.0, 56.5, 83.0],
    "North_East_Atlantic" => [-42.0, -0.1, 24.9, 48.0],
    "Baltic_Sea" => [9.4, 30.9, 53.0, 65.9],
    "Black_Sea" => [26.5, 41.95, 40.0, 47.95],
    "Mediterranean_Sea" => [-7.0, 36.375, 30.0, 45.875],
    "North_Sea" => [-100.0, 50.0, -80.0, 80.0],
    "Canary_Islands" => [-20.0, -9.0, 25.0, 31.5],
    "World_Ocean" => [-180.0, 180.0, -90.0, 90.0],
);

In [5]:
begin
    mindepth = 0.0 #Minimum water depth
    maxdepth = 1000.0 #Maximum water depth
    datestart = DateTime(1960, 1, 1)
    dateend = DateTime(2024, 12, 31)
end

2024-12-31T00:00:00

In [6]:
variableunits = Dict(
    "sea_water_temperature" => "degree_Celsius",
    "sea_water_salinity" => "psu",
    "mass_concentration_of_chlorophyll_a_in_sea_water" => "mg/m3",
    "moles_of_nitrate_per_unit_mass_in_sea_water" => "micromole/kg",
);

## Health check

In [7]:
for datasource in datasourcelist
    r = HTTP.get(
        joinpath(DIVAndFairEase.beacon_services[datasource], "api/health"),
        ["Authorization" => "Bearer $(token)"],
    )
    if r.status == 200
        @info("$(datasource) Beacon instance is working")
    else
        @warn("$(datasource) Beacon is down or not reachable.")
    end

end

[ Info: Euro-Argo Beacon instance is working
[ Info: CORA Profile Beacon instance is working
[ Info: CORA Timeseries Beacon instance is working
[ Info: World Ocean Database Beacon instance is working
[ Info: SeaDataNet CDI TS Beacon instance is working


In [8]:
minlon = domaininfo[regionname][1]
maxlon = domaininfo[regionname][2]
minlat = domaininfo[regionname][3]
maxlat = domaininfo[regionname][4]

outputfilelist = []
for datasource in datasourcelist

    @info("Working on $(datasource)")

    parameter1 = "TEMP"
    if datasource == "World Ocean Database"
        parameter1 = "Temperature"
    elseif datasource == "EMODnet Chemistry"
        parameter1 = "ITS_90_water_temperature"
    elseif datasource == "SeaDataNet CDI TS"
        parameter1 = "TEMPPR01"
    end
    parameter2 = "PSAL"

    datasource_name = replace(datasource, " " => "-")

    # Build query
    query = DIVAndFairEase.prepare_query(
        datasource,
        parameter1,
        Dates.Date(datestart),
        Dates.Date(dateend),
        mindepth,
        maxdepth,
        minlon,
        maxlon,
        minlat,
        maxlat,
    )

    # Construct file and figure names
    filename = joinpath(
        datadir,
        "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.nc",
    )

    # Write the data in netCDF files
    @info("Data will be written in file:\n$(filename)")
    if isfile(filename)
        @info("File already downloaded")

    else
        @time open(filename, "w") do io
            r = HTTP.request(
                "POST",
                joinpath(DIVAndFairEase.beacon_services[datasource], "api/query"),
                [
                    "Content-type" => "application/json",
                    "Authorization" => "Bearer $(token)",
                ],
                query,
                response_stream = io,
            )
            @info(r.status)
        end

        NCDataset(filename, "a") do ds
            ds.attrib["data_source"] = datasource
            ds.attrib["date_created"] = Dates.format(Dates.now(), "yyyy-mm-ddTHH:MM:SS")
            # Maybe add other attributes
        end
    end
    @info("NetCDF file size: $(round(filesize(filename)/1000^2, digits=1))M")
    push!(outputfilelist, filename)

end

[ Info: Working on Euro-Argo
┌ Info: Data will be written in file:
└ ../data/North_Adriatic/North_Adriatic_Euro-Argo_TEMP_19600101-20241231_0-1000m.nc
[ Info: File already downloaded
[ Info: NetCDF file size: 3.0M
[ Info: Working on CORA Profile
[ Info: Working with CORA dataset
┌ Info: Data will be written in file:
└ ../data/North_Adriatic/North_Adriatic_CORA-Profile_TEMP_19600101-20241231_0-1000m.nc
[ Info: File already downloaded
[ Info: NetCDF file size: 14.2M
[ Info: Working on CORA Timeseries
[ Info: Working with CORA dataset
┌ Info: Data will be written in file:
└ ../data/North_Adriatic/North_Adriatic_CORA-Timeseries_TEMP_19600101-20241231_0-1000m.nc
[ Info: File already downloaded
[ Info: NetCDF file size: 53.4M
[ Info: Working on World Ocean Database
┌ Info: Data will be written in file:
└ ../data/North_Adriatic/North_Adriatic_World-Ocean-Database_Temperature_19600101-20241231_0-1000m.nc
[ Info: File already downloaded
[ Info: NetCDF file size: 71.4M
[ Info: Working on SeaData

### Make plot

In [9]:
if doplot
    if length(outputfilelist) > 0
        colorlist = [:red, :blue, :green, :orange, :seagreen1, :aqua, :magenta]

        fig = Figure(size = (800, 500))
        ga = GeoAxis(
            fig[1, 1];
            dest = "+proj=merc",
            title = "$(regionname) observations\n($(Dates.Date(datestart)) - $(Dates.Date(dateend)))",
        )

        for (iii, outputfile) in enumerate(outputfilelist)
            NCDataset(outputfile, "r") do ds
                datasource = ds.attrib["data_source"]
                lon = ds["LONGITUDE"][:]
                lat = ds["LATITUDE"][:]
                plot!(
                    ga,
                    lon,
                    lat,
                    color = colorlist[iii],
                    markersize = 2,
                    label = "$(datasource) ($(length(lon)) obs.)",
                )
            end

        end
        lines!(ga, GeoMakie.coastlines(10), color = :black)
        # poly!(ga, land, color=:grey)
        xlims!(ga, minlon, maxlon)
        ylims!(ga, minlat, maxlat)
        fig[1, 2] = Legend(fig, ga, "Data source")

        figname1 = joinpath(
            figdir,
            "$(regionname)_$(replace(varname, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.png",
        )
        Makie.save(figname1, fig)

        fig
    else
        @warn("No file has been written, hence no plot will be produced")
    end
end

In [62]:
if true
    for (iii, outputfile) in enumerate(outputfilelist)
        NCDataset(outputfile, "r") do ds
            datasource = ds.attrib["data_source"]
            lon = ds["LONGITUDE"][:]
            lat = ds["LATITUDE"][:]
            obsvalues = coalesce.(ds[keys(ds)[1]][:], NaN)
            thevar = ds[keys(ds)[1]][:]
            goodvalues = findall(.!(ismissing.(thevar)))
            lonall[iii] = ds["LONGITUDE"][goodvalues]
            latall[iii] = ds["LATITUDE"][goodvalues]
            depthall[iii] = ds["DEPTH"][goodvalues]
            timesall[iii] = coalesce.(ds["datetime"][goodvalues], NaN)

            if typeof(thevar[goodvalues]) <: Vector{Union{Missing,String}}
                obsvalues = parse.(Float64, thevar[goodvalues])
            else
                obsvalues = thevar[goodvalues]
            end

            fig = Figure(size = (800, 500))
            ax = Axis(
                fig[1, 1],
                title = "$(regionname) observation histogram\n($(Dates.Date(datestart)) - $(Dates.Date(dateend)))",
            )
            hist!(ax, obsvalues)

            figname1 = joinpath(
                figdir,
                "histogram_$(regionname)_$(replace(varname, " "=>"_"))_$(datasource)_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.png",
            )
            Makie.save(figname1, fig)
            fig
        end
    end
end

In [10]:
if doplot
    fig2 = Figure(size = (800, 800))

    for (iii, outputfile) in enumerate(outputfilelist)
        NCDataset(outputfile, "r") do ds
            datasource = ds.attrib["data_source"]
            lon = ds["LONGITUDE"][:]
            lat = ds["LATITUDE"][:]
            ga = GeoAxis(
                fig2[1+(iii-1)÷2, 1+(iii-1)%2];
                dest = "+proj=merc",
                title = "$(datasource) ($(length(lon)) obs.)",
            )
            plot!(ga, lon, lat, color = :orange, markersize = 2)
            lines!(ga, GeoMakie.coastlines(10), color = :black)
            # poly!(ga, land, color=:grey)
            xlims!(ga, minlon, maxlon)
            ylims!(ga, minlat, maxlat)
        end

    end

    figname2 = joinpath(
        figdir,
        "$(regionname)_$(replace(varname, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m_all.png",
    )
    save(figname2, fig2)
    fig2
end

## Duplicate removal
### Set parameters

In [11]:
Δlon = 0.01
Δlat = 0.01
Δdepth = 0.05
Δtime = 0.0
Δvar = 0.01

0.01

In [12]:
# Allocate vectors of vectors
nfiles = length(outputfilelist)
lonall = Vector{Vector{Float64}}(undef, nfiles)
latall = Vector{Vector{Float64}}(undef, nfiles)
timesall = Vector{Vector{DateTime}}(undef, nfiles)
depthall = Vector{Vector{Float64}}(undef, nfiles)
obsall = Vector{Vector{Float64}}(undef, nfiles)
obsidall = Vector{Vector{String}}(undef, nfiles)

for (iii, datafile) in enumerate(outputfilelist)
    @info("Working on file $(basename(datafile))")

    # Set the reference date (different for each dataset)
    dateref = Dates.DateTime(1950, 1, 1)
    if occursin("World_Ocean_Database", basename(datafile))
        dateref = Dates.DateTime(1770, 1, 1)
    elseif occursin("EMODnet Chemistry", basename(datafile))
        dateref = Dates.DateTime(1921, 1, 1)
    end

    NCDataset(datafile, "r") do ds

        # Get coordinates
        thevar = ds[keys(ds)[1]][:]
        goodvalues = findall(.!(ismissing.(thevar)))
        lonall[iii] = ds["LONGITUDE"][goodvalues]
        latall[iii] = ds["LATITUDE"][goodvalues]
        depthall[iii] = ds["DEPTH"][goodvalues]
        timesall[iii] = coalesce.(ds["datetime"][goodvalues], NaN)

        if typeof(thevar[goodvalues]) <: Vector{Union{Missing,String}}
            obsall[iii] = parse.(Float64, thevar[goodvalues])
        else
            obsall[iii] = thevar[goodvalues]
        end
    end
end

[ Info: Working on file North_Adriatic_Euro-Argo_TEMP_19600101-20241231_0-1000m.nc
[ Info: Working on file North_Adriatic_CORA-Profile_TEMP_19600101-20241231_0-1000m.nc
[ Info: Working on file North_Adriatic_CORA-Timeseries_TEMP_19600101-20241231_0-1000m.nc
[ Info: Working on file North_Adriatic_World-Ocean-Database_Temperature_19600101-20241231_0-1000m.nc
[ Info: Working on file North_Adriatic_SeaDataNet-CDI-TS_TEMPPR01_19600101-20241231_0-1000m.nc


### Remove duplicates

In [13]:
@time obslon, obslat, obsdepth, obstime, obsval, pct = DIVAndFairEase.merge_datasets2(
    lonall,
    latall,
    depthall,
    timesall,
    obsall,
    Δlon,
    Δlat,
    Δdepth,
    Δtime,
    Δvar,
);

[ Info: Working on dataset #2
[ Info: 30970
[ Info: Number of possible duplicates: 0
[ Info: Percentage of duplicates: 0.0%
[ Info: Working on dataset #3
[ Info: 461374
[ Info: Number of possible duplicates: 0
[ Info: Percentage of duplicates: 0.0%
[ Info: Working on dataset #4
[ Info: 2105014
[ Info: Number of possible duplicates: 82247
[ Info: Percentage of duplicates: 30.1%
[ Info: Working on dataset #5
[ Info: 2296046


 52.544324 seconds (26.05 M allocations: 19.243 GiB, 7.20% gc time, 0.62% compilation time)


[ Info: Number of possible duplicates: 0
[ Info: Percentage of duplicates: 0.0%


In [14]:
begin
    fig2 = Figure(size = (800, 500))
    ga2 = GeoAxis(fig2[1, 1]; dest = "+proj=merc", title = "$(regionname) observations")

    plot!(
        ga2,
        obslon,
        obslat,
        color = :orange,
        markersize = 5,
        label = "$(length(obslon)) obs.",
    )
    lines!(ga2, GeoMakie.coastlines(10), color = :black)
    xlims!(ga2, minlon, maxlon)
    ylims!(ga2, minlat, maxlat)
    fig2[1, 2] = Legend(fig2, ga2, "Data source")

    figname3 = joinpath(
        figdir,
        "$(regionname)_$(replace(varname, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m_dupl.png",
    )
    save(figname3, fig2)
end

CairoMakie.Screen{IMAGE}


In [34]:
obsid = [randstring(8) for ii = 1:length(obslon)];

3994752-element Vector{String}:
 "CMcxBpB4"
 "PWO6PeUl"
 "WRD4q22L"
 "CH0olXOp"
 "xXVBVVwK"
 "PWUwIzWZ"
 "SNc1ARSm"
 "oPm1wo0L"
 "wcUku23s"
 "7mapC4Ay"
 "i8v4naLQ"
 "hngmmmsw"
 "wZj4m9zQ"
 ⋮
 "YiKNRbLG"
 "EovU9XIs"
 "k4GuXYh1"
 "1tIDsTLr"
 "MDAdQUGv"
 "kNw1XIEB"
 "RBCSt2h2"
 "fb32tTXl"
 "b0BFb2SZ"
 "GwQ8KUxa"
 "10cyy8ja"
 "W1UPWLPb"

In [35]:
outputfile = joinpath(outputdir, "$(regionname)_merged.nc")
isfile(outputfile) ? rm(outputfile) : @debug("ok")
DIVAnd.saveobs(outputfile, varname, obsval, (obslon, obslat, obsdepth, obstime), obsid)

NCDataset(outputfile, "a") do ds
    ds.attrib["data_source"] = datasourcelist
    ds.attrib["date_created"] = Dates.format(Dates.now(), "yyyy-mm-ddTHH:MM:SS")
    # Maybe add other attributes
end

"2025-06-02T10:06:00"

## 3. DIVAnd interpolation

In [36]:
regionname = "North_Adriatic"
dx, dy = 0.05, 0.05
lonr = minlon:dx:maxlon
latr = minlat:dy:maxlat
timerange = [Date(1960, 1, 1), Date(2024, 12, 31)];

depthr = [
    0.0,
    5.0,
    10.0,
    15.0,
    20.0,
    25.0,
    30.0,
    40.0,
    50.0,
    60,
    70,
    80,
    90,
    100,
    125,
    150,
    175,
    200,
];

yearlist = [1960:2024];
monthlist = 1:12;

TS = DIVAnd.TimeSelectorYearListMonthList(yearlist, monthlist)
@info("$(length(TS)) time periods")

[ Info: 12 time periods


In [37]:
begin
    datadir = "../data/"
    bathfile = joinpath(datadir, "gebco_30sec_4.nc")
    if isfile(bathfile)
        @info("Bathymetry already downloaded")
    else
        Downloads.download(
            "https://dox.uliege.be/index.php/s/RSwm4HPHImdZoQP/download",
            bathfile,
        )
    end

    bx, by, b = load_bath(bathfile, true, lonr, latr)
    _, _, mask = load_mask(bathfile, true, lonr, latr, depthr)
end

[ Info: Bathymetry already downloaded


(12.0:0.05:18.0, 43.0:0.05:46.0, Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0;;; 0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0;;; 0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0;;; … ;;; 0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0;;; 0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0;;; 0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0])

In [38]:
if doplot
    figbathy = GeoMakie.Figure()
    ga1 = GeoAxis(figbathy[1, 1]; dest = "+proj=merc", title = "GEBCO bathymetry")
    hm1 = heatmap!(
        ga1,
        bx,
        by,
        -b,
        interpolate = false,
        colorrange = (-400.0, 400),
        colormap = :delta,
    )
    xlims!(ga1, (bx[1], bx[end]))
    ylims!(ga1, (by[1], by[end]))
    contour!(ga1, bx, by, b, levels = [0.0], color = :black)
    Colorbar(figbathy[2, 1], hm1, vertical = false, label = "(m)")
    figbathy
end

In [39]:
"""
	plot_mask(mask, depth)

Plot the land-sea mask at the selected depth
"""
function plot_mask(mask, depth)
    NN = 1
    fig = GeoMakie.Figure()
    figtitle = "Land-sea mask at depth $(depth) m"
    ga = GeoAxis(fig[1, 1]; dest = "+proj=merc", title = figtitle)
    hm = GeoMakie.heatmap!(ga, bx, by, mask, colormap = Reverse(:binary))
    GeoMakie.xlims!(ga, (bx[1], bx[end]))
    GeoMakie.ylims!(ga, (by[1], by[end]))
    return fig, ga
end

plot_mask

In [40]:
if doplot
    fig, ga = plot_mask(mask[:, :, 1], depthr[1])
    fig
end

In [41]:
label = DIVAnd.floodfill(mask)

if doplot

    fig2 = Figure()
    ga2 = GeoAxis(fig2[1, 1]; dest = "+proj=merc", title = "Flood fill")
    hm2 = heatmap!(ga2, bx, by, label[:, :, 1], interpolate = false)
    xlims!(ga2, (bx[1], bx[end]))
    ylims!(ga2, (by[1], by[end]))
    Colorbar(fig2[2, 1], hm2, vertical = false)
    fig2
end

In [42]:
maskfilled = label .== 1
if doplot
    fig, ga = plot_mask(maskfilled[:, :, 1], depthr[1])
    fig
end

In [43]:
obsval, obslon, obslat, obsdepth, obstime, obsid = loadobs(Float64, outputfile, varname);

In [44]:
if doplot
    fig5, ga5 = plot_mask(maskfilled[:, :, 1], depthr[1])
    ga5.title = "Observations and land-sea mask"
    plot!(ga5, obslon, obslat, markersize = 2, color = :orange)
    fig5
end

In [45]:
# @time rdiag=1.0./DIVAnd.weight_RtimesOne((obslon,obslat),(0.03,0.03));
# @show maximum(rdiag),mean(rdiag)

In [47]:
begin
    sz = (length(lonr), length(latr), length(depthr))
    lenx = fill(100_000.0, sz)   # 100 km
    leny = fill(100_000.0, sz)   # 100 km
    lenz = fill(25.0, sz)      # 25 m 
    len = (lenx, leny, lenz)
    epsilon2 = 0.1
    # epsilon2 = epsilon2 * rdiag;
end

0.1

In [48]:
begin
    metadata = OrderedDict(
        # Name of the project (SeaDataCloud, SeaDataNet, EMODNET-chemistry, ...)
        "project" => "FAIR-EASE",

        # URN code for the institution EDMO registry,
        # e.g. SDN:EDMO::1579
        "institution_urn" => "SDN:EDMO::1579",

        # Production group
        "production" => "GHER, ULiège",

        # Name and emails from authors
        "Author_e-mail" => ["Charles Troupin <ctroupin@uliege.be"],

        # Source of the observation
        "source" => "observational data from SeaDataNet and World Ocean Atlas",

        # Additional comment
        "comment" => "Duplicate removal applied to the merged dataset",

        # SeaDataNet Vocabulary P35 URN
        # http://seadatanet.maris2.nl/v_bodc_vocab_v2/search.asp?lib=p35
        # example: SDN:P35::WATERTEMP
        "parameter_keyword_urn" => "SDN:P35::EPC00001",

        # List of SeaDataNet Parameter Discovery Vocabulary P02 URNs
        # http://seadatanet.maris2.nl/v_bodc_vocab_v2/search.asp?lib=p02
        # example: ["SDN:P02::TEMP"]
        "search_keywords_urn" => ["SDN:P02::PSAL"],

        # List of SeaDataNet Vocabulary C19 area URNs
        # SeaVoX salt and fresh water body gazetteer (C19)
        # http://seadatanet.maris2.nl/v_bodc_vocab_v2/search.asp?lib=C19
        # example: ["SDN:C19::3_1"]
        "area_keywords_urn" => ["SDN:C19::3_3"],
        "product_version" => "1.0",
        "product_code" => "something-to-decide",

        # bathymetry source acknowledgement
        # see, e.g.
        # * EMODnet Bathymetry Consortium (2016): EMODnet Digital Bathymetry (DTM).
        # https://doi.org/10.12770/c7b53704-999d-4721-b1a3-04ec60c87238
        # 
        # taken from
        # http://www.emodnet-bathymetry.eu/data-products/acknowledgement-in-publications
        #
        # * The GEBCO Digital Atlas published by the British Oceanographic Data Centre on behalf of IOC and IHO, 2003
        #
        # taken from
        # https://www.bodc.ac.uk/projects/data_management/international/gebco/gebco_digital_atlas/copyright_and_attribution/

        "bathymetry_source" => "The GEBCO Digital Atlas published by the British Oceanographic Data Centre on behalf of IOC and IHO, 2003",

        # NetCDF CF standard name
        # http://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html
        # example "standard_name" = "sea_water_temperature",
        "netcdf_standard_name" => "sea_water_temperature",
        "netcdf_long_name" => "sea water temperature",
        "netcdf_units" => "1e-3",

        # Abstract for the product
        "abstract" => "...",

        # This option provides a place to acknowledge various types of support for the
        # project that produced the data
        "acknowledgement" => "...",
        "documentation" => "https://doi.org/doi_of_doc",

        # Digital Object Identifier of the data product
        "doi" => "...",
    )
end

OrderedDict{String, Any} with 19 entries:
  "project"               => "FAIR-EASE"
  "institution_urn"       => "SDN:EDMO::1579"
  "production"            => "GHER, ULiège"
  "Author_e-mail"         => ["Charles Troupin <ctroupin@uliege.be"]
  "source"                => "observational data from SeaDataNet and World Ocea…
  "comment"               => "Duplicate removal applied to the merged dataset"
  "parameter_keyword_urn" => "SDN:P35::EPC00001"
  "search_keywords_urn"   => ["SDN:P02::PSAL"]
  "area_keywords_urn"     => ["SDN:C19::3_3"]
  "product_version"       => "1.0"
  "product_code"          => "something-to-decide"
  "bathymetry_source"     => "The GEBCO Digital Atlas published by the British …
  "netcdf_standard_name"  => "sea_water_temperature"
  "netcdf_long_name"      => "sea water temperature"
  "netcdf_units"          => "1e-3"
  "abstract"              => "..."
  "acknowledgement"       => "..."
  "documentation"         => "https://doi.org/doi_of_doc"
  "doi"            

In [49]:
begin
    outputfile = joinpath(outputdir, "$(regionname)_$(varname)")
    isfile(outputfile) ? rm(outputfile) : @debug("ok")

    @time dbinfo = diva3d(
        (lonr, latr, depthr, TS),
        (obslon, obslat, obsdepth, obstime),
        obsval,
        len,
        epsilon2,
        outputfile,
        "temperature",
        bathname = bathfile,
        mask = maskfilled,
        fitcorrlen = false,
        niter_e = 2,
        surfextend = true,
    )

end

[ Info: Creating netCDF file ../output/AdriaticV1.nc
[ Info: Time step 1 / 12
[ Info: scaled correlation length (min,max) in dimension 1: (100000.0, 100000.0)
[ Info: scaled correlation length (min,max) in dimension 2: (100000.0, 100000.0)
[ Info: scaled correlation length (min,max) in dimension 3: (25.0, 25.0)
┌ Warning: resolution (17.5) is too coarse for correlation length 25.0 in dimension 3 at indices CartesianIndex(49, 1, 15) (skipping further tests). It is recommended that the resolution is at least 2 times finer than the correlation length.
└ @ DIVAnd ~/.julia/packages/DIVAnd/4UymR/src/utils.jl:18
[ Info: number of windows: 8
┌ Warning: resolution (17.5) is too coarse for correlation length 25.0 in dimension 3 at indices CartesianIndex(49, 1, 15) (skipping further tests). It is recommended that the resolution is at least 2 times finer than the correlation length.
└ @ DIVAnd ~/.julia/packages/DIVAnd/4UymR/src/utils.jl:18
[ Info: number of windows: 8
[ Info: Time step 2 / 12
[ In

116.435359 seconds (112.25 M allocations: 163.431 GiB, 25.29% gc time, 32.34% compilation time)


Dict{Symbol, Any} with 5 entries:
  :used               => Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1…
  :residuals          => [5.27379e6, -9.261e5, 2.51874e5, 1.37376e6, 4.39711e6,…
  :mask               => Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0;;;…
  :factore            => [88.7781 311.795 … 386.583 278.502; 613.528 916.556 … …
  :background_profile => [-2.28521e5 -1.46253e5 … -2.80192e5 -2.0427e5; -2.5326…

("North_Adriatic", "sea_water_temperature")